# VeloceReduction -- Tutorial

This tutorial provides an example on how to reduce data of a given night YYMMDD.

In [1]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
from astropy.io import fits
from pathlib import Path
import sys
import argparse

# VeloceReduction modules and function
from VeloceReduction import config
from VeloceReduction.utils import identify_calibration_and_science_runs, polynomial_function
from VeloceReduction.extraction import extract_orders, extract_initial_order_ranges_and_coeffs
from VeloceReduction.calibration import calibrate_wavelength

## Adjust Date and Directory (possibly via argument parser)

In [2]:
def parse_arguments():
    # Create the parser
    parser = argparse.ArgumentParser(description="Process some inputs.")
    
    # Add arguments
    parser.add_argument('-d','--date', type=str, default="001122",
                        help='Date in the format DDMMYY (e.g., "001122")')
    parser.add_argument('-wd','--working_directory', type=str, default="./",
                        help='The directory where the script will operate.')
    
    # Parse the arguments
    args = parser.parse_args()
    return args

def get_script_input():
    if 'ipykernel' in sys.modules:
        # Assume default values if inside Jupyter
        jupyter_date = "001122"
        jupyter_working_directory = "./"
        print("Running in a Jupyter notebook. Using predefined values")
        args = argparse.Namespace(date=jupyter_date, working_directory=jupyter_working_directory)
    else:
        # Use argparse to handle command-line arguments
        print("Running as a standalone Python script")
        args = parse_arguments()

    return args

# Use the function to get input
args = get_script_input()
config.date = args.date
config.working_directory = args.working_directory
print(f"Date: {args.date}, Working Directory: {args.working_directory}")

Running in a Jupyter notebook. Using predefined values
Date: 001122, Working Directory: ./


## Identfiy Calibration and Science Runs

In [3]:
# Extract the Calibration and Science data from the night log
calibration_runs, science_runs = identify_calibration_and_science_runs(config.date, config.working_directory+'observations/')



Identifying calibration and science runs now

Found Log file ./observations//001122/001122-AAT-example.log



## Extract orders and save in initial FITS files with an extension per order.

In [4]:
# Extract initial order ranges and coefficients
initial_order_ranges, initial_order_coeffs = extract_initial_order_ranges_and_coeffs()

In [5]:
# # Extract Master Flat
# print('Extracting Master Flat')
# master_flat, noise = extract_orders(
#     ccd1_runs = calibration_runs['Flat_60.0'][:1],
#     ccd2_runs = calibration_runs['Flat_1.0'][:1],
#     ccd3_runs = calibration_runs['Flat_0.1'][:1],
#     Flat = True
# )

# # Extract Master ThXe
# print('Extracting Master ThXe')
# master_thxe, noise = extract_orders(
#     ccd1_runs = calibration_runs['FibTh_180.0'][:1],
#     ccd2_runs = calibration_runs['FibTh_60.0'][:1],
#     ccd3_runs = calibration_runs['FibTh_15.0'][:1]
# )

# # Extract Master LC
# print('Extracting Master LC')
# master_lc, noise = extract_orders(
#     ccd1_runs = calibration_runs['SimLC'][-1:],
#     ccd2_runs = calibration_runs['SimLC'][-1:],
#     ccd3_runs = calibration_runs['SimLC'][-1:],
#     LC = True,
#     # tramline_debug = True
# )

In [6]:
# # Extract Science Objects and save them into FITS files under reduced_data/
# for science_object in list(science_runs.keys()):
#     print('Extracting '+science_object)
#     try:
#         science, science_noise = extract_orders(
#             ccd1_runs = science_runs[science_object],
#             ccd2_runs = science_runs[science_object],
#             ccd3_runs = science_runs[science_object],
#             Science=True
#         )

#         # Create a primary HDU and HDU list
#         primary_hdu = fits.PrimaryHDU()
#         hdul = fits.HDUList([primary_hdu])

#         # Loop over your extension names and corresponding data arrays
#         for ext_index, ext_name in enumerate(initial_order_coeffs):
#             # Create an ImageHDU object for each extension

#             # Define the columns with appropriate formats
#             col1_def = fits.Column(name='wave_vac',format='E', array=np.arange(len(science[ext_index,:]),dtype=float))
#             col2_def = fits.Column(name='wave_air',format='E', array=np.arange(len(science[ext_index,:]),dtype=float))
#             col3_def = fits.Column(name='science', format='E', array=science[ext_index,:])
#             col4_def = fits.Column(name='science_noise',   format='E', array=science_noise[ext_index,:])
#             col5_def = fits.Column(name='flat',    format='E', array=master_flat[ext_index,:])
#             col6_def = fits.Column(name='thxe',    format='E', array=master_thxe[ext_index,:])
#             col7_def = fits.Column(name='lc',      format='E', array=master_lc[ext_index,:])

#             hdu = fits.BinTableHDU.from_columns([col1_def, col2_def, col3_def, col4_def, col5_def, col6_def, col7_def], name=science_object+'_'+ext_name.lower())

#             # Append the HDU to the HDU list
#             hdul.append(hdu)

#         # Save to a new FITS file with an extension for each order
#         Path(config.working_directory+'reduced_data/'+config.date+'/'+science_object).mkdir(parents=True, exist_ok=True)    
#         hdul.writeto(config.working_directory+'reduced_data/'+config.date+'/'+science_object+'/veloce_spectra_'+science_object+'_'+config.date+'.fits', overwrite=True)

#         print('Successfully extracted '+science_object)

#     except:
#         print('Failed to extract '+science_object)

## Wavelength calibration

In [7]:
for science_object in list(science_runs.keys()):
    try:
        calibrate_wavelength(science_object, create_overview_pdf=True)
        print('Succesfully calibrated wavelength for '+science_object)
    except:
        print('Failed to calibrate wavelength for '+science_object)

1 ccd_1_order_138
[4383.472  4383.506  4383.54   ... 4485.1377 4485.1543 4485.1704]
2 ccd_1_order_139
[4352.1094 4352.1416 4352.1743 ... 4453.7812 4453.799  4453.8164]
3 ccd_1_order_140
[4320.486  4320.52   4320.554  ... 4420.5347 4420.5483 4420.5625]
4 ccd_1_order_141
[4289.936  4289.969  4290.0024 ... 4389.9185 4389.934  4389.9497]
5 ccd_1_order_142
[4259.148  4259.182  4259.216  ... 4359.163  4359.1787 4359.195 ]
6 ccd_1_order_143
[4229.7764 4229.809  4229.8423 ... 4328.5435 4328.559  4328.574 ]
7 ccd_1_order_144
[4200.352  4200.3853 4200.418  ... 4298.3975 4298.412  4298.4272]
8 ccd_1_order_145
[4171.3286 4171.361  4171.3936 ... 4268.833  4268.848  4268.8633]
9 ccd_1_order_146
[4142.3926 4142.426  4142.459  ... 4239.4536 4239.4688 4239.4834]
10 ccd_1_order_147
[4114.213  4114.2456 4114.2783 ... 4210.513  4210.528  4210.5425]
11 ccd_1_order_148
[4086.4988 4086.531  4086.5632 ... 4182.2563 4182.271  4182.286 ]
12 ccd_1_order_149
[4059.0525 4059.0845 4059.1162 ... 4154.031  4154.0454 

99 ccd_3_order_95
[6362.055  6362.1064 6362.157  ... 6513.361  6513.386  6513.411 ]
100 ccd_3_order_96
[6295.3687 6295.4204 6295.472  ... 6443.903  6443.9233 6443.9443]
101 ccd_3_order_97
[6231.755  6231.8013 6231.848  ... 6379.8286 6379.856  6379.8833]
102 ccd_3_order_98
[6167.171  6167.22   6167.269  ... 6313.9062 6313.93   6313.9536]
103 ccd_3_order_99
[6104.519  6104.5684 6104.6177 ... 6250.066  6250.089  6250.112 ]
104 ccd_3_order_100
[6043.187  6043.236  6043.2847 ... 6187.7363 6187.7593 6187.7827]
105 ccd_3_order_101
[5983.4834 5983.5317 5983.5796 ... 6126.5664 6126.5894 6126.6123]
106 ccd_3_order_102
[5924.389  5924.4375 5924.4854 ... 6066.4517 6066.474  6066.4966]
107 ccd_3_order_103
[5869.6245 5869.6636 5869.703  ... 6008.9683 6008.995  6009.022 ]
108 ccd_3_order_104
[5810.258  5810.304  5810.35   ... 5949.8755 5949.898  5949.9204]
Succesfully calibrated wavelength for HIP69673
